In [ ]:
# to-do: clean this up
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import scipy
from scipy.optimize import minimize as op
from astropy.stats import LombScargle
import scipy.ndimage
from scipy.fftpack import fft,ifft,fftfreq
import pickle

In [ ]:
pickleload = open("rawdata.pickle","rb")
data = pickle.load(pickleload)

In [ ]:
# set first guess at mixer properties; the better this is to start, the better!
#amp0 = 2000.
f0 = 14.187637 #(November 18, 2019) cycles per day CAREFULLY TUNED BY HAND
#f0 = 13.43635 # cycles per day NOT YET CAREFULLY TUNED BY HAND!
#phase0 = -1.0 # chosen arbitrarily
#bandwidth = .2 # cycles per day

In [ ]:
quarters = np.unique(np.sort(data["quarter"]))
print(quarters)

In [ ]:
for quarter in quarters:
    I = data["quarter"] == quarter
    # bandpass filter to a narrow frequency range
    thisflux = (data["flux"])[I]
    thistime = (data["time"])[I]
    thisy = fft(thisflux)
    thisq = fftfreq(len(thisflux)) / np.nanmedian(thistime[1:] - thistime[:-1])
    notband = np.abs(np.abs(thisq) - f0) > 3. * bandwidth
    thisy[notband] = 0.
    thisflux = ifft(thisy).real
    data["flux"][I] = thisflux

In [ ]:
plt.figure(figsize=(15,5))
plt.scatter(data["time"],data["flux"],marker=".",alpha=0.1, c=data["quarter"])
plt.xlabel("time",fontsize=20)
plt.ylabel("flux",fontsize=20)

In [ ]:
# HACK: In this block, the number 50 is a magic number
# Instead it should be the inverse bandwidth / the nanmedian of time differences or something like that.
#amp0=1202.78 #phase0=-1.15

In [ ]:
mixer = amp0 * np.exp(1j * (2 * np.pi * f0 * data["time"] - phase0))
mf = mixer * data["flux"]
immf = mf.imag
remf = mf.real
sremf = scipy.ndimage.filters.gaussian_filter1d(remf, 50)
simmf = scipy.ndimage.filters.gaussian_filter1d(immf, 50)

In [ ]:
A=np.mean(immf**2 + remf**2)
phi=np.arctan2(np.mean(immf),np.mean(remf))
def objective(f,A,phi,data):
    remf=mf.real
    immf=mf.imag
    return np.var(immf)
f=op(objective,f0,args=(A,phi,data))

In [ ]:
print(f)

In [ ]:
plt.scatter(remf, immf, s=5, alpha=0.25)
print(np.mean(remf),np.mean(immf))

In [ ]:
# tune the radio
# that is, optimize amp0, f0, phase0 to get a concentrated blob at (1, 0)

In [ ]:
plt.plot(data["time"], sremf, "b-", alpha=0.5)
plt.plot(data["time"], simmf, "r-", alpha=0.5)

In [ ]:
#need to work on grey band
plt.figure(figsize=(15,5))
plt.plot(data["time"],data["flux"])
plt.xlabel("time",fontsize=20)
plt.ylabel("flux",fontsize=20)
plt.title("ZOOMED INSERT TITLE")
plt.xlim(130,170)

In [ ]:
data["remf"] = remf
data["immf"] = immf
data["sremf"] = sremf
data["simmf"] = simmf
data["f0"] = f0
data["phase0"] = phase0
data["bandwidth"] = bandwidth
data["amp0"] = amp0

In [ ]:
picklesave = open("filtered_and_mixed.pickle","wb")
pickle.dump(data, picklesave)
picklesave.close()